In [1]:
    import torch.nn as nn
    import torch
    import numpy as np
    import gym

    import ipynb.fs.full.misc as misc
    import ipynb.fs.full.breeder as br
    import ipynb.fs.full.mutation_functions as mf
    import ipynb.fs.full.selection_functions as sf
    import ipynb.fs.full.crossover_functions as cf
    from ipynb.fs.full.agent_class import Agent
    import ipynb.fs.full.net_class as nm

In [2]:
class PendulumAgentNet(nm.NetModule):
    def __init__(self):
        super(PendulumAgentNet, self).__init__()

        self.num_inputs = 3
        self.num_outputs = 1

        #  define net structure here
        self.model = nn.Sequential(
            nn.Linear(self.num_inputs, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, self.num_outputs),
            nn.Tanh(),
        )

#         self.model = self.model.cuda()

        self.parameters_count = self.count_parameters()

    def forward(self, inputs):
        ret = self.model(inputs)
        return ret

In [3]:
class PendulumAgent(Agent):
    def __init__(self, dna=None):
        super(PendulumAgent, self).__init__(dna, PendulumAgentNet())

    def step(self, observation):
        state = np.asarray(observation)

        torch_state = misc.single_to_torch(state, cuda=self.net.is_cuda())
        torch_action = self.net(torch_state)
        action = misc.single_from_torch(torch_action, cuda=self.net.is_cuda())

        self.add_experience(state, action)

        return action
    
    def evaluate(self, environment, render=False, sleep=False, idx=0):
        avg_fitness = 0.0
        tries = 2
        for _ in range(tries):
            avg_fitness += super(PendulumAgent, self).evaluate(environment, render=render, sleep=sleep, idx=idx)
        
        return avg_fitness / tries

    @staticmethod
    def produce_offspring(dna):
        return PendulumAgent(dna=dna)

In [4]:
env = gym.make("Pendulum-v0")

population_size = 50
population = [PendulumAgent()] * population_size

breeder = br.Breeder(population,
                     env,
                     -5.,
                     generations=200,
                     elite_percent=0.1,
                     mutation_rate=0.3,
                     selection_pressure=0.2,
                     selection_func=sf.tournament,
                     crossover_func=cf.none,
                     mutation_func=mf.sm_g_sum_r,
                     render=False)


breeder.load_checkpoint()
agent = breeder.evolve()

for _ in range(10):
    agent.evaluate(env, render=True, sleep=True)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Gen: 0 , Max fitness: -1.423424770763053 , Min fitness: -1687.5141603305967 , Age: 0 , Mean fitness: -1005.6459803713353
